# ETL — RAW → SILVER (Sinistros 2025) usando DDL (PostgreSQL)

Este notebook:
1. Lê o CSV **RAW** (`datatran2025.csv`)
2. Aplica as transformações definidas para a **SILVER**
3. Executa o **DDL** para criar `silver.silver_sinistros` no PostgreSQL
4. Carrega os dados na tabela (modo padrão: **reload**)

> Rodar no **VS Code** (kernel local). PostgreSQL no Docker.

## 0) Dependências

```bash
python -m pip install pandas numpy sqlalchemy psycopg2-binary
```

## 1) Conexão com PostgreSQL

In [31]:
import os
import pandas as pd
import numpy as np
from pathlib import Path
from sqlalchemy import create_engine, text

# CONFIG (VS Code / fora do Docker)
os.environ["POSTGRES_HOST"] = os.getenv("POSTGRES_HOST", "127.0.0.1")
os.environ["POSTGRES_PORT"] = os.getenv("POSTGRES_PORT", "5433")   
os.environ["POSTGRES_DB"] = os.getenv("POSTGRES_DB", "sinistros_2025")
os.environ["POSTGRES_USER"] = os.getenv("POSTGRES_USER", "postgres")
os.environ["POSTGRES_PASSWORD"] = os.getenv("POSTGRES_PASSWORD", "dan1920")

user = os.environ["POSTGRES_USER"]
pwd  = os.environ["POSTGRES_PASSWORD"]
host = os.environ["POSTGRES_HOST"]
port = os.environ["POSTGRES_PORT"]
db   = os.environ["POSTGRES_DB"]

engine = create_engine(f"postgresql+psycopg2://{user}:{pwd}@{host}:{port}/{db}", pool_pre_ping=True)
with engine.connect() as conn:
    conn.execute(text("SELECT 1"))
print(f"Conectado! host={host} port={port} db={db} user={user}")

Conectado! host=127.0.0.1 port=5433 db=sinistros_2025 user=postgres


## 2) Ler CSV RAW

In [32]:
def find_raw_csv() -> Path:
    # padrão do projeto
    p = Path.cwd()
    for _ in range(10):
        candidate = p / "Data Layer" / "raw" / "datatran2025.csv"
        if candidate.exists():
            return candidate
        if p == p.parent:
            break
        p = p.parent

    # fallback (mesma pasta do notebook)
    candidate = Path("datatran2025.csv")
    if candidate.exists():
        return candidate

    raise FileNotFoundError(
        "Não encontrei o CSV RAW. Coloque em 'Data Layer/raw/datatran2025.csv' "
        "ou ajuste a função find_raw_csv()."
    )

RAW_PATH = find_raw_csv()
RAW_PATH

WindowsPath('c:/Users/Daniel/OneDrive/Documentos/SINISTROS2025/Data Layer/raw/datatran2025.csv')

In [33]:
df = pd.read_csv(RAW_PATH, encoding="latin-1", sep=";", low_memory=False)
df.shape

(65683, 30)

In [34]:
df.head(3)

,id,data_inversa,dia_semana,horario,uf,br,km,municipio,causa_acidente,tipo_acidente,...,feridos_graves,ilesos,ignorados,feridos,veiculos,latitude,longitude,regional,delegacia,uop
0,652493,2025-01-01,quarta-feira,06:20:00,SP,116,225,GUARULHOS,Reação tardia ou ineficiente do condutor,Tombamento,...,0,0,1,1,2,"-23,48586772","-46,54075317",SPRF-SP,DEL01-SP,UOP01-DEL01-SP
1,652519,2025-01-01,quarta-feira,07:50:00,CE,116,"546,2",PENAFORTE,Pista esburacada,Colisão frontal,...,0,1,4,1,6,"-7,812288","-39,08333306",SPRF-CE,DEL05-CE,UOP03-DEL05-CE
2,652522,2025-01-01,quarta-feira,08:45:00,PR,369,"88,2",CORNELIO PROCOPIO,Reação tardia ou ineficiente do condutor,Colisão traseira,...,0,2,0,3,2,"-23,182565","-50,637228",SPRF-PR,DEL07-PR,UOP05-DEL07-PR


## 3) Transformações RAW → SILVER (contrato atual)

In [35]:
# 3.1 Remover colunas (como combinado)
DROP_COLS = ["km", "feridos_leves", "feridos_graves", "ignorados", "regional", "delegacia", "uop"]
df = df.drop(columns=[c for c in DROP_COLS if c in df.columns], errors="ignore")

# 3.2 Renomes
RENAME = {"data_inversa":"data_acidente", "horario":"hora_acidente", "uso_solo":"area_urbana"}
df = df.rename(columns={k:v for k,v in RENAME.items() if k in df.columns})

# 3.3 Padronização básica de texto (strip + UPPER)
text_cols = df.select_dtypes(include=["object"]).columns.tolist()
for c in text_cols:
    df[c] = df[c].astype("string").str.strip().str.upper()

# 3.4 area_urbana: SIM/NÃO -> boolean
def sim_nao_to_bool(s: pd.Series) -> pd.Series:
    s = s.astype("string").str.strip().str.upper()
    return s.map({
        "SIM": True, "S": True, "TRUE": True,
        "NÃO": False, "NAO": False, "N": False, "FALSE": False
    }).astype("boolean")

if "area_urbana" in df.columns:
    df["area_urbana"] = sim_nao_to_bool(df["area_urbana"])

# 3.5 Numéricos (negativos -> NULL)
for c in [c for c in ["pessoas","mortos","ilesos","feridos","veiculos"] if c in df.columns]:
    df[c] = pd.to_numeric(df[c], errors="coerce")
    df.loc[df[c] < 0, c] = np.nan
    df[c] = df[c].round(0).astype("Int64")

# id
if "id" in df.columns:
    df["id"] = pd.to_numeric(df["id"], errors="coerce").astype("Int64")

# 3.6 Latitude/Longitude (vírgula->ponto + range)
if "latitude" in df.columns:
    df["latitude"] = pd.to_numeric(df["latitude"].astype("string").str.replace(",", ".", regex=False), errors="coerce")
    df.loc[~df["latitude"].between(-90, 90), "latitude"] = np.nan

if "longitude" in df.columns:
    df["longitude"] = pd.to_numeric(df["longitude"].astype("string").str.replace(",", ".", regex=False), errors="coerce")
    df.loc[~df["longitude"].between(-180, 180), "longitude"] = np.nan

# 3.7 Datas e horas (sem data_hora_acidente)
if "data_acidente" in df.columns:
    df["data_acidente"] = pd.to_datetime(df["data_acidente"], dayfirst=True, errors="coerce").dt.date

if "hora_acidente" in df.columns:
    h = df["hora_acidente"].astype("string")
    t1 = pd.to_datetime(h, format="%H:%M", errors="coerce")
    t2 = pd.to_datetime(h, format="%H:%M:%S", errors="coerce")
    df["hora_acidente"] = t1.fillna(t2).dt.time

df.shape

C:\Users\Daniel\AppData\Local\Temp\ipykernel_12160\688353504.py:10: Pandas4Warning: For backward compatibility, 'str' dtypes are included by select_dtypes when 'object' dtype is specified. This behavior is deprecated and will be removed in a future version. Explicitly pass 'str' to `include` to select them, or to `exclude` to remove them and silence this warning.
See https://pandas.pydata.org/docs/user_guide/migration-3-strings.html#string-migration-select-dtypes for details on how to write code that works with pandas 2 and 3.
  text_cols = df.select_dtypes(include=["object"]).columns.tolist()


(65683, 23)

In [36]:
df.dtypes

id                          Int64
data_acidente              object
dia_semana                 string
hora_acidente              object
uf                         string
br                          int64
municipio                  string
causa_acidente             string
tipo_acidente              string
classificacao_acidente     string
fase_dia                   string
sentido_via                string
condicao_metereologica     string
tipo_pista                 string
tracado_via                string
area_urbana               boolean
pessoas                     Int64
mortos                      Int64
ilesos                      Int64
feridos                     Int64
veiculos                    Int64
latitude                  Float64
longitude                 Float64
dtype: object

## 4) Executar o DDL (criar tabela SILVER)

In [37]:
DDL_PATH = Path(r"C:\Users\Daniel\OneDrive\Documentos\SINISTROS2025\Data Layer\silver\ddl.sql")
ddl_sql = DDL_PATH.read_text(encoding="utf-8")
print("DDL carregado:", DDL_PATH)

DDL carregado: C:\Users\Daniel\OneDrive\Documentos\SINISTROS2025\Data Layer\silver\ddl.sql


In [38]:
# Executa múltiplas statements DDL via conexão raw do psycopg2
raw = engine.raw_connection()
try:
    raw.autocommit = True
    cur = raw.cursor()
    cur.execute(ddl_sql)
    cur.close()
finally:
    raw.close()

print("DDL executado com sucesso.")

DDL executado com sucesso.


## 5) Carregar dados na tabela (append)

In [39]:
TARGET_SCHEMA = "silver"
TARGET_TABLE  = "silver_sinistros"

# Puxa colunas reais do banco (pra evitar mismatch)
cols_in_db = pd.read_sql(
    """
    SELECT column_name
    FROM information_schema.columns
    WHERE table_schema='silver' AND table_name='silver_sinistros'
    ORDER BY ordinal_position
    """,
    engine
)["column_name"].tolist()

df_load = df.copy()
df_load = df_load[[c for c in df_load.columns if c in cols_in_db]]

# Se id é PK no DDL, remover linhas sem id
if "id" in cols_in_db and "id" in df_load.columns:
    before = len(df_load)
    df_load = df_load[df_load["id"].notna()].copy()
    print("Linhas sem id removidas:", before - len(df_load))

df_load.to_sql(
    name=TARGET_TABLE,
    con=engine,
    schema=TARGET_SCHEMA,
    if_exists="append",
    index=False,
    chunksize=5000,
    method="multi",
)

print(f"OK - carregado em {TARGET_SCHEMA}.{TARGET_TABLE} | linhas tentadas={len(df_load)}")

Linhas sem id removidas: 0
OK - carregado em silver.silver_sinistros | linhas tentadas=65683


## 6) Validação

In [40]:
check = pd.read_sql(
    """
    SELECT
      COUNT(*)::bigint AS linhas,
      MIN(data_acidente) AS inicio,
      MAX(data_acidente) AS fim,
      COALESCE(SUM(mortos),0)::bigint AS mortos_total,
      COALESCE(SUM(feridos),0)::bigint AS feridos_total
    FROM silver.silver_sinistros
    """,
    engine
)
check

,linhas,inicio,fim,mortos_total,feridos_total
0,131366,2025-01-01,2025-12-11,10932,150756


In [41]:
pd.read_sql("SELECT * FROM silver.silver_sinistros LIMIT 10", engine)

,id,data_acidente,dia_semana,hora_acidente,uf,br,municipio,causa_acidente,tipo_acidente,classificacao_acidente,...,tipo_pista,tracado_via,area_urbana,pessoas,mortos,ilesos,feridos,veiculos,latitude,longitude
0,652493,2025-01-01,QUARTA-FEIRA,06:20:00,SP,116,GUARULHOS,REAÇÃO TARDIA OU INEFICIENTE DO CONDUTOR,TOMBAMENTO,COM VÍTIMAS FERIDAS,...,MÚLTIPLA,RETA;DECLIVE,True,2,0,0,1,2,-23.485868,-46.540753
1,652519,2025-01-01,QUARTA-FEIRA,07:50:00,CE,116,PENAFORTE,PISTA ESBURACADA,COLISÃO FRONTAL,NaN,...,SIMPLES,RETA,False,6,1,1,1,6,-7.812288,-39.083333
2,652522,2025-01-01,QUARTA-FEIRA,08:45:00,PR,369,CORNELIO PROCOPIO,REAÇÃO TARDIA OU INEFICIENTE DO CONDUTOR,COLISÃO TRASEIRA,COM VÍTIMAS FERIDAS,...,DUPLA,RETA;ACLIVE,True,5,0,2,3,2,-23.182565,-50.637228
3,652544,2025-01-01,QUARTA-FEIRA,11:00:00,PR,116,CAMPINA GRANDE DO SUL,REAÇÃO TARDIA OU INEFICIENTE DO CONDUTOR,SAÍDA DE LEITO CARROÇÁVEL,COM VÍTIMAS FERIDAS,...,DUPLA,RETA,False,5,0,4,1,2,-25.365177,-49.042230
4,652549,2025-01-01,QUARTA-FEIRA,09:30:00,MG,251,FRANCISCO SA,VELOCIDADE INCOMPATÍVEL,COLISÃO FRONTAL,COM VÍTIMAS FERIDAS,...,SIMPLES,CURVA;DECLIVE,False,5,0,1,2,4,-16.468013,-43.431213
5,652569,2025-01-01,QUARTA-FEIRA,10:40:00,MT,70,CACERES,TRANSITAR NA CONTRAMÃO,COLISÃO FRONTAL,COM VÍTIMAS FATAIS,...,SIMPLES,RETA,False,4,2,1,0,5,-16.041486,-57.258840
6,652573,2025-01-01,QUARTA-FEIRA,12:23:00,RS,116,TAPES,AUSÊNCIA DE REAÇÃO DO CONDUTOR,SAÍDA DE LEITO CARROÇÁVEL,COM VÍTIMAS FERIDAS,...,DUPLA,RETA,False,2,0,0,1,2,-30.739714,-51.625940
7,652617,2025-01-01,QUARTA-FEIRA,17:45:00,SC,101,SAO JOSE,AUSÊNCIA DE REAÇÃO DO CONDUTOR,COLISÃO TRASEIRA,COM VÍTIMAS FERIDAS,...,DUPLA,RETA,True,2,0,1,1,2,-27.600012,-48.622647
8,652625,2025-01-01,QUARTA-FEIRA,18:40:00,MG,116,MURIAE,VELOCIDADE INCOMPATÍVEL,TOMBAMENTO,COM VÍTIMAS FATAIS,...,SIMPLES,CURVA,False,2,1,0,0,2,-21.163289,-42.379690
9,652648,2025-01-01,QUARTA-FEIRA,17:00:00,PE,407,AFRANIO,DEMAIS FALHAS MECÂNICAS OU ELÉTRICAS,INCÊNDIO,SEM VÍTIMAS,...,SIMPLES,ACLIVE;CURVA,False,2,0,2,0,1,-8.475031,-41.013711
